# Exact Method
Using Gurobi solver

In [1]:
import math
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [2]:
df = pd.read_csv('datasets/real-datasets/compas_metadata.csv').drop(columns = ['instances'])
performance_columns = [col for col in df.columns if col.startswith('algo')]
X = df.drop(columns = performance_columns)
Y = df[performance_columns][['algo_bagging']]
F = X.to_numpy()
Y = Y.to_numpy()

# Como pelo jeito não consigo modelar nosso problema matricial, vamos pegar uma parte de F e Y
F = np.array(F[0,0])
y = np.array(Y[0][0])

In [3]:

try:

    # Create a new model
    mod = gp.Model("mod1") 

    # Create variables
    Ar = mod.addVar(vtype=GRB.CONTINUOUS, name="Ar") 
    Br = mod.addVar(vtype=GRB.CONTINUOUS, name="Br")
    cr = mod.addVar(vtype=GRB.CONTINUOUS, name="cr")
    
    # Set objective
    mod.setObjective( (F - Br*Ar*F) + (y - cr*Ar*F), GRB.MINIMIZE
                    )
    
    # Adjust parameters
    mod.params.NonConvex = 2

    # Optimize model
    mod.optimize()
    
    # print variables
    for v in mod.getVars():
        print('%s %g' % (v.varName, v.x))
        
    #print optimized objective function
    print('Obj: %g' % mod.objVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Encountered an attribute error')
    print(e.message)

Restricted license - for non-production use only - expires 2022-01-13
Changed value of parameter NonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 0 rows, 3 columns and 0 nonzeros
Model fingerprint: 0x07f362a7
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]

Continuous model is non-convex -- solving as a MIP.

Found heuristic solution: objective 1.6006911
Found heuristic solution: objective -6.53809e+18
Presolve time: 0.00s
Presolved: 5 rows, 7 columns, 11 nonzeros
Presolved model has 2 bilinear constraint(s)
Variable types: 7 continuous, 0 integer (0 binary)

Root relaxation: unbounded, 3 iterations, 0.00 seconds

    Nodes    |    Current Node   